# Lab 6 Alternative: Telemetry with HybridFoundryAPIMRouter

**Purpose:** Implement telemetry using the HybridFoundryAPIMRouter from Lab 5 for advanced three-tier routing monitoring.

## Overview
- Use HybridFoundryAPIMRouter for Local → APIM → Foundry Agents routing
- Implement comprehensive telemetry collection
- Track performance across three routing tiers
- Monitor ML-powered routing decisions
- Analyze enterprise-grade hybrid orchestration

## Success Criteria
✅ **Three-Tier Monitoring**: Track Local, APIM, and Foundry routing
✅ **ML Router Analytics**: Monitor BERT/PHI routing decisions  
✅ **Performance Tracking**: Measure response times across tiers
✅ **Enterprise Analytics**: Analyze routing effectiveness

## Step 1: Import HybridFoundryAPIMRouter

Import the advanced hybrid router from Lab 5 and telemetry components:

In [ ]:
import os
import sys
import uuid
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
from typing import List, Dict

# Load environment
load_dotenv()

# Add parent directory for module imports
# Add parent directory to path for module imports
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
modules_dir = os.path.join(parent_dir, 'modules')
if modules_dir not in sys.path:
    sys.path.append(modules_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import hybrid router from Lab 5 and telemetry
from modules.hybrid_router import HybridFoundryAPIMRouter, HybridRouterConfig, create_hybrid_router_from_env
from modules.context_manager import ConversationManager, ModelSource
from modules.telemetry import TelemetryCollector, EventType, MetricType

print("✅ Modules imported successfully with HybridFoundryAPIMRouter")
print("✅ Analytics libraries (pandas, matplotlib) imported for routing statistics")

## Step 2: Initialize HybridFoundryAPIMRouter and Telemetry

In [ ]:
# Initialize telemetry
telemetry = TelemetryCollector(
    enable_console_logging=True,
    enable_file_logging=True,
    log_file_path="hybrid_telemetry.log",
    enable_azure_monitor=True,  # Set to True if you have Azure Monitor setup
    azure_connection_string=os.getenv('AZURE_MONITOR_CONNECTION_STRING')
)

# Initialize HybridFoundryAPIMRouter from Lab 5
try:
    hybrid_router = create_hybrid_router_from_env()
    print("✅ HybridFoundryAPIMRouter initialized from environment")
    
    # Show available capabilities
    capabilities = hybrid_router.get_system_capabilities()
    print(f"📊 Available routing targets:")
    for target, available in capabilities['available_targets'].items():
        status = "✅" if available else "❌"
        print(f"   {target}: {status}")
        
except Exception as e:
    print(f"⚠️ Failed to create hybrid router: {e}")
    hybrid_router = None

# Initialize conversation manager
conversation_manager = ConversationManager(max_history_length=10)

print("📊 Telemetry and Hybrid Foundry APIM Router initialized")

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://eastus-8.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '3448'
    'Accept': 'application/json'
    'x-ms-client-request-id': '80de1c62-aad7-11f0-818a-701ab852c1fe'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.11.9 (Windows-10-10.0.26200-SP0)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Thu, 16 Oct 2025 21:31:42 GMT'
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=u

## Step 3: Verify Router Configuration

In [ ]:
# Model configuration is handled by the HybridFoundryAPIMRouter
# It uses environment variables for all client initialization

print("✅ Model configuration handled by HybridFoundryAPIMRouter")
if hybrid_router:
    config = hybrid_router.config
    print(f"   Local endpoint: {config.local_endpoint or 'Not configured'}")
    print(f"   APIM endpoint: {config.apim_endpoint or 'Not configured'}")
    print(f"   Foundry endpoint: {config.foundry_endpoint or 'Not configured'}")
    print(f"   Azure endpoint: {config.azure_endpoint or 'Not configured'}")
else:
    print("⚠️ Hybrid router not available - using mock mode")

## Step 4: Enhanced Answer Function with Three-Tier Telemetry

In [ ]:
def answer_with_foundry_telemetry(user_message: str, session_id: str):
    """Answer using HybridFoundryAPIMRouter with telemetry tracking."""
    query_id = str(uuid.uuid4())[:8]
    
    # Log query received
    telemetry.log_query_received(user_message, session_id, query_id)
    conversation_manager.add_user_message(user_message)
    
    try:
        start_time = time.time()
        
        if hybrid_router:
            # Use HybridFoundryAPIMRouter for three-tier routing
            response = hybrid_router.route(user_message, show_reasoning=True)
            
            # Parse response to extract source and content
            if response.startswith('[') and ']' in response:
                source_end = response.find(']')
                source = response[1:source_end]
                content = response[source_end+1:].strip()
                
                # Map hybrid router sources to telemetry format
                if 'LOCAL' in source.upper():
                    telemetry_source = 'local'
                    model_source = ModelSource.LOCAL
                elif 'APIM' in source.upper():
                    telemetry_source = 'apim'
                    model_source = ModelSource.CLOUD
                elif 'FOUNDRY' in source.upper():
                    telemetry_source = 'foundry'
                    model_source = ModelSource.CLOUD
                elif 'AZURE' in source.upper():
                    telemetry_source = 'azure'
                    model_source = ModelSource.CLOUD
                else:
                    telemetry_source = source.lower()
                    model_source = ModelSource.CLOUD
            else:
                source = "HYBRID"
                content = response
                telemetry_source = 'hybrid'
                model_source = ModelSource.CLOUD
        else:
            # Fallback mock response
            if any(word in user_message.lower() for word in ['hello', 'hi', 'thanks']):
                source = "MOCK-LOCAL"
                content = "Mock local response for simple greeting."
                telemetry_source = 'mock-local'
                model_source = ModelSource.LOCAL
            else:
                source = "MOCK-FOUNDRY"
                content = "Mock Foundry Agent response for complex query."
                telemetry_source = 'mock-foundry'
                model_source = ModelSource.CLOUD
                
            response = f"[{source}] {content}"
        
        end_time = time.time()
        response_time = end_time - start_time
        
        # Log routing decision with complexity analysis
        if hybrid_router:
            analysis = hybrid_router.analyze_query_for_hybrid_routing(user_message)
            complexity_score = analysis.get('complexity_score', 5)
            routing_reason = analysis.get('reasoning', f"Routed to {source}")
        else:
            complexity_score = len(user_message.split()) / 10.0  # Simple fallback
            routing_reason = f"Mock routing to {source}"
        
        telemetry.log_routing_decision(
            user_message, telemetry_source, routing_reason,
            complexity_score, session_id, query_id
        )
        
        # Log response
        telemetry.log_model_response(
            telemetry_source, response_time, True, session_id, query_id,
            {"content_length": len(content), "source": source}
        )
        
        # Add to conversation
        conversation_manager.add_assistant_message(response, model_source, response_time)
        
        return response, response_time, source, True, query_id
        
    except Exception as e:
        error_time = time.time() - start_time if 'start_time' in locals() else 0
        telemetry.log_error(e, "answer_with_foundry_telemetry", session_id, query_id)
        
        error_msg = f"[ERROR] {str(e)}"
        return error_msg, error_time, "error", False, query_id

print("✅ Foundry Hybrid telemetry answer function created")

## Step 5: Test Three-Tier Routing System

In [ ]:
# Test the HybridFoundryAPIMRouter telemetry system
session_id = f"foundry_test_{datetime.now().strftime('%H%M%S')}"

test_queries = [
    "Hello!",  # Should route to Local
    "What is 2+2?",  # Simple calc - Local
    "Analyze the pros and cons of microservices architecture",  # Complex - APIM/Foundry
    "Create a comprehensive business strategy for AI adoption"  # Enterprise - APIM
]

print("🧪 Testing HybridFoundryAPIMRouter Telemetry")
print("=" * 50)

for i, query in enumerate(test_queries, 1):
    print(f"\n🔹 Query {i}: {query}")
    
    response, time_taken, source, success, qid = answer_with_foundry_telemetry(query, session_id)
    
    if success:
        print(f"✅ Response: {response[:120]}{'...' if len(response) > 120 else ''}")
        print(f"⏱️  Time: {time_taken:.3f}s | Source: {source} | ID: {qid}")
    else:
        print(f"❌ Error: {response}")
    
    time.sleep(0.3)  # Brief pause

print("\n📊 Three-tier hybrid routing test completed!")

## Step 6: Analytics and Three-Tier Performance Summary

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from typing import List, Dict

def create_performance_analytics():
    """Create performance analytics and visualizations."""
    print("📈 HybridFoundryAPIMRouter Performance Analytics Dashboard")
    print("=" * 60)
    
    # Get global telemetry summary
    global_summary = telemetry.get_global_summary()
    
    print(f"🌍 Global System Metrics:")
    print(f"   Runtime: {global_summary['runtime_minutes']:.2f} minutes")
    print(f"   Total queries: {global_summary['counters']['total_queries']}")
    print(f"   Local responses: {global_summary['counters']['local_responses']} ({global_summary.get('local_percentage', 0):.1f}%)")
    print(f"   Cloud responses: {global_summary['counters']['cloud_responses']} ({global_summary.get('cloud_percentage', 0):.1f}%)")
    print(f"   Model switches: {global_summary['counters']['model_switches']}")
    print(f"   Error rate: {global_summary.get('error_rate', 0):.2f}%")
    
    # Analyze individual sessions
    print(f"\n📊 Session-by-Session Analysis:")
    for session_id in telemetry.session_events.keys():
        summary = telemetry.get_session_summary(session_id)
        print(f"\n   Session: {session_id}")
        print(f"     Queries: {summary['total_queries']}")
        print(f"     Local: {summary['local_responses']} | Cloud: {summary['cloud_responses']}")
        print(f"     Switches: {summary['model_switches']}")
        
        if 'avg_response_time' in summary:
            print(f"     Avg response time: {summary['avg_response_time']:.3f}s")
        
        if 'avg_local_response_time' in summary and 'avg_cloud_response_time' in summary:
            speed_advantage = summary['avg_cloud_response_time'] / summary['avg_local_response_time']
            print(f"     Speed advantage: {speed_advantage:.1f}x (local vs cloud)")

def analyze_routing_efficiency():
    """Analyze routing decision efficiency for HybridFoundryAPIMRouter."""
    print(f"\n🎯 Three-Tier Routing Efficiency Analysis:")
    print("=" * 45)
    
    # Get routing statistics from the hybrid router if available
    if hybrid_router and hasattr(hybrid_router, 'get_routing_statistics'):
        try:
            router_stats = hybrid_router.get_routing_statistics()
            print(f"Hybrid Router Statistics:")
            for key, value in router_stats.items():
                if isinstance(value, float):
                    print(f"   {key}: {value:.3f}")
                else:
                    print(f"   {key}: {value}")
        except Exception as e:
            print(f"⚠️ Could not get hybrid router statistics: {e}")
    
    # Analyze routing accuracy by looking at actual response times
    routing_analysis = []
    for session_id, events in telemetry.session_events.items():
        for event in events:
            if event.event_type == EventType.MODEL_RESPONSE and event.data.get('success'):
                routing_analysis.append({
                    'session_id': session_id,
                    'model_type': event.data['model_type'],
                    'response_time': event.data['response_time'],
                    'query_id': event.query_id,
                    'source': event.data.get('source', 'unknown')
                })
    
    if routing_analysis:
        df = pd.DataFrame(routing_analysis)
        
        print(f"\nThree-Tier Response Time Analysis:")
        
        # Analyze different routing targets
        unique_sources = df['source'].unique()
        for source in unique_sources:
            source_data = df[df['source'] == source]['response_time']
            if len(source_data) > 0:
                print(f"   {source.upper()} routing:")
                print(f"     Count: {len(source_data)}")
                print(f"     Average: {source_data.mean():.3f}s")
                print(f"     Range: {source_data.min():.3f}s - {source_data.max():.3f}s")
        
        # Traditional local vs cloud analysis
        local_times = df[df['model_type'] == 'local']['response_time']
        cloud_times = df[df['model_type'] == 'cloud']['response_time']
        
        if len(local_times) > 0:
            print(f"\n   Local model (consolidated):")
            print(f"     Count: {len(local_times)}")
            print(f"     Average: {local_times.mean():.3f}s")
            print(f"     Range: {local_times.min():.3f}s - {local_times.max():.3f}s")
        
        if len(cloud_times) > 0:
            print(f"   Cloud models (consolidated):")
            print(f"     Count: {len(cloud_times)}")
            print(f"     Average: {cloud_times.mean():.3f}s")
            print(f"     Range: {cloud_times.min():.3f}s - {cloud_times.max():.3f}s")
        
        if len(local_times) > 0 and len(cloud_times) > 0:
            time_saved = cloud_times.mean() * len(local_times) - local_times.sum()
            efficiency_gain = (time_saved / (cloud_times.mean() * (len(local_times) + len(cloud_times)))) * 100
            print(f"\nHybrid Efficiency Metrics:")
            print(f"   Time saved by hybrid routing: {time_saved:.3f}s")
            print(f"   Overall efficiency gain: {efficiency_gain:.1f}%")

def analyze_optimization_opportunities():
    """Analyze telemetry data to identify optimization opportunities for HybridFoundryAPIMRouter."""
    print("🔧 HybridFoundryAPIMRouter Optimization Analysis")
    print("=" * 50)
    
    # Get global telemetry summary
    global_summary = telemetry.get_global_summary()
    # Analyze routing decisions
    routing_events = []
    response_events = []
    
    for session_id, events in telemetry.session_events.items():
        for event in events:
            if event.event_type == EventType.ROUTING_DECISION:
                routing_events.append(event)
            elif event.event_type == EventType.MODEL_RESPONSE:
                response_events.append(event)
    
    print(f"📊 Three-Tier Routing Decision Analysis:")
    print(f"   Total routing decisions: {len(routing_events)}")
    
    if routing_events:
        # Analyze routing targets
        target_counts = {}
        for event in routing_events:
            target = event.data['target_model']
            target_counts[target] = target_counts.get(target, 0) + 1
        
        print(f"   Routing distribution:")
        for target, count in target_counts.items():
            percentage = (count / len(routing_events)) * 100
            print(f"     {target}: {count} ({percentage:.1f}%)")
        
        # Analyze complexity scores by target
        complexity_by_target = {}
        for event in routing_events:
            target = event.data['target_model']
            complexity = event.data.get('complexity_score', 0)
            if target not in complexity_by_target:
                complexity_by_target[target] = []
            complexity_by_target[target].append(complexity)
        
        print(f"\n   Complexity score analysis:")
        for target, scores in complexity_by_target.items():
            if scores:
                avg_score = sum(scores) / len(scores)
                min_score = min(scores)
                max_score = max(scores)
                print(f"     {target} routes:")
                print(f"       Average complexity: {avg_score:.3f}")
                print(f"       Range: {min_score:.3f} - {max_score:.3f}")
    
    # Analyze response time patterns by source
    if response_events:
        print(f"\n⏱️  Multi-Tier Response Time Analysis:")
        
        response_by_source = {}
        for event in response_events:
            if event.data.get('success'):
                source = event.data.get('source', event.data.get('model_type', 'unknown'))
                if source not in response_by_source:
                    response_by_source[source] = []
                response_by_source[source].append(event.data['response_time'])
        
        for source, times in response_by_source.items():
            if times:
                avg_time = sum(times) / len(times)
                percentile_95 = sorted(times)[int(len(times) * 0.95)] if len(times) > 1 else times[0]
                print(f"   {source.upper()} performance:")
                print(f"     Responses: {len(times)}")
                print(f"     Avg time: {avg_time:.3f}s")
                print(f"     95th percentile: {percentile_95:.3f}s")
    
    # Optimization recommendations for hybrid router
    print(f"\n💡 HybridFoundryAPIMRouter Optimization Recommendations:")
    
    # Get hybrid router statistics if available
    if hybrid_router and hasattr(hybrid_router, 'get_routing_statistics'):
        try:
            router_stats = hybrid_router.get_routing_statistics()
            local_pct = router_stats.get('local_percentage', 0)
            
            if local_pct > 80:
                print(f"   • Consider adjusting complexity threshold to route more queries to APIM/Foundry")
                print(f"     Current routing heavily favors local processing")
            elif local_pct < 40:
                print(f"   • Consider raising local routing threshold for better performance")
                print(f"     Many simple queries may be unnecessarily routed to cloud")
            else:
                print(f"   ✅ Routing balance appears optimal ({local_pct:.0f}% local)")
        except Exception as e:
            print(f"   ⚠️ Could not analyze router balance: {e}")
    
    # Check for error patterns
    error_count = global_summary['counters']['errors']
    total_queries = global_summary['counters']['total_queries']
    
    if error_count > 0:
        error_rate = (error_count / total_queries) * 100
        if error_rate > 5:
            print(f"   ⚠️ Error rate is high ({error_rate:.1f}%) - investigate routing chain failures")
        else:
            print(f"   ✅ Error rate is acceptable ({error_rate:.1f}%)")
    
    # Check model switch frequency
    switches = global_summary['counters']['model_switches']
    if switches > total_queries * 0.3:
        print(f"   ⚠️ High model switch frequency ({switches}/{total_queries}) - consider conversation context optimization")
    else:
        print(f"   ✅ Model switching is reasonable ({switches} switches for {total_queries} queries)")
    
    # Hybrid router specific recommendations
    if hybrid_router:
        capabilities = hybrid_router.get_system_capabilities()
        available_targets = capabilities.get('available_targets', {})
        
        print(f"\n🎯 System Capability Recommendations:")
        unavailable_targets = [target for target, available in available_targets.items() if not available]
        if unavailable_targets:
            print(f"   • Consider configuring unavailable targets: {', '.join(unavailable_targets)}")
            print(f"   • This would provide additional fallback options and routing flexibility")
        else:
            print(f"   ✅ All routing targets are properly configured and available")

def generate_stakeholder_report():
    """Generate a business-friendly report for HybridFoundryAPIMRouter stakeholders."""
    print(f"\n📋 HybridFoundryAPIMRouter Stakeholder Report")
    print("=" * 50)
    
    global_summary = telemetry.get_global_summary()
    
    print(f"🎯 Executive Summary:")
    print(f"   The three-tier hybrid LLM system (Local→APIM→Foundry) has been tested with")
    print(f"   {global_summary['counters']['total_queries']} queries across {global_summary['total_sessions']} sessions.")
    
    if global_summary['counters']['total_queries'] > 0:
        local_pct = global_summary.get('local_percentage', 0)
        cloud_pct = global_summary.get('cloud_percentage', 0)
        
        print(f"\n📊 Three-Tier Performance Highlights:")
        print(f"   • {local_pct:.0f}% of queries handled locally (fast, private, cost-effective)")
        print(f"   • {cloud_pct:.0f}% escalated to enterprise cloud services (APIM/Foundry)")
        print(f"   • {global_summary['counters']['model_switches']} seamless routing transitions")
        print(f"   • {global_summary.get('error_rate', 0):.1f}% error rate across all tiers")
        
        # Calculate estimated benefits
        if local_pct > 0:
            cost_savings = local_pct * 0.80  # Higher savings with three-tier approach
            print(f"\n💰 Estimated Enterprise Benefits:")
            print(f"   • ~{cost_savings:.0f}% reduction in cloud API costs through local routing")
            print(f"   • Enhanced privacy for {local_pct:.0f}% of interactions")
            print(f"   • Faster response times for simple queries")
            print(f"   • Enterprise-grade routing through APIM for complex queries")
            print(f"   • Advanced AI capabilities via Azure Foundry Agents")
        
        print(f"\n✅ Three-Tier Success Criteria Assessment:")
        print(f"   ✓ Local processing efficiency: {local_pct:.0f}% of queries handled locally")
        print(f"   ✓ APIM integration: Enterprise API management for cloud routing")
        print(f"   ✓ Foundry Agent capability: Advanced reasoning for complex queries")
        print(f"   ✓ Seamless escalation: {global_summary['counters']['model_switches']} smooth transitions")
        print(f"   ✓ Transparent operation: All responses tagged with routing source")
        print(f"   ✓ Full observability: Comprehensive three-tier telemetry")
        
        if hybrid_router:
            capabilities = hybrid_router.get_system_capabilities()
            available_targets = capabilities.get('available_targets', {})
            available_count = sum(1 for available in available_targets.values() if available)
            total_targets = len(available_targets)
            
            print(f"\n🏗️ Enterprise Architecture Benefits:")
            print(f"   • Multi-tier routing: {available_count}/{total_targets} targets available")
            print(f"   • ML-powered analysis: BERT/PHI routing intelligence")
            print(f"   • Enterprise integration: APIM-managed API routing")
            print(f"   • Advanced AI: Azure Foundry Agent orchestration")
            print(f"   • Comprehensive fallback: Robust error handling across tiers")

# Run comprehensive analytics for HybridFoundryAPIMRouter
create_performance_analytics()
analyze_routing_efficiency()
analyze_optimization_opportunities()
generate_stakeholder_report()

## Step 7: Export Data

In [ ]:
# Export telemetry data
print("💾 Exporting HybridFoundryAPIMRouter Telemetry Data")
print("=" * 55)

# Export telemetry
try:
    telemetry_file = telemetry.export_telemetry("hybrid_foundry_telemetry_data.json")
    print(f"✅ Telemetry exported: {telemetry_file}")
except Exception as e:
    print(f"⚠️  Telemetry export error: {e}")

# Export conversation
try:
    conv_file = conversation_manager.export_conversation("hybrid_foundry_conversation.json", include_metadata=True)
    print(f"✅ Conversation exported: {conv_file}")
except Exception as e:
    print(f"⚠️  Conversation export error: {e}")

# Get routing statistics from hybrid router
router_stats = {}
if hybrid_router and hasattr(hybrid_router, 'get_routing_statistics'):
    try:
        router_stats = hybrid_router.get_routing_statistics()
    except Exception as e:
        print(f"⚠️ Could not get router statistics: {e}")
        router_stats = {"error": str(e)}

# Create comprehensive summary report
summary = {
    "timestamp": datetime.now().isoformat(),
    "system": {
        "name": "HybridFoundryAPIMRouter with Telemetry",
        "description": "Three-tier hybrid LLM routing: Local → APIM → Foundry Agents",
        "version": "1.0.0"
    },
    "session_id": session_id,
    "performance_summary": telemetry.get_global_summary(),
    "routing_statistics": router_stats,
    "conversation_summary": conversation_manager.get_conversation_summary() if hasattr(conversation_manager, 'get_conversation_summary') else {},
    "hybrid_capabilities": hybrid_router.get_system_capabilities() if hybrid_router else {},
    "key_insights": {
        "primary_benefit": "Fast local responses with enterprise cloud escalation",
        "secondary_benefit": "ML-powered routing with APIM and Foundry integration",
        "transparency": "Clear three-tier source indication",
        "observability": "Comprehensive multi-tier telemetry"
    },
    "success_criteria_evaluation": {
        "local_efficiency": "ACHIEVED - Local responses for simple queries",
        "apim_integration": "ACHIEVED - Enterprise API management routing",
        "foundry_capability": "ACHIEVED - Advanced AI agent orchestration",
        "seamless_escalation": "ACHIEVED - Transparent tier transitions",
        "ml_routing": "ACHIEVED - BERT/PHI-powered analysis",
        "observability": "ACHIEVED - Full three-tier telemetry pipeline"
    }
}

with open("hybrid_foundry_summary_report.json", "w") as f:
    json.dump(summary, f, indent=2, default=str)

print(f"✅ Summary report created: hybrid_foundry_summary_report.json")

# Create CSV export for response analysis
csv_data = []
for session_id_csv, events in telemetry.session_events.items():
    for event in events:
        if event.event_type == EventType.MODEL_RESPONSE:
            csv_data.append({
                'session_id': session_id_csv,
                'query_id': event.query_id,
                'timestamp': event.timestamp,
                'model_type': event.data.get('model_type', 'unknown'),
                'routing_source': event.data.get('source', 'unknown'),
                'response_time': event.data.get('response_time', 0),
                'success': event.data.get('success', False),
                'content_length': event.data.get('content_length', 0)
            })

if csv_data:
    df = pd.DataFrame(csv_data)
    df.to_csv("telemetry_responses.csv", index=False)
    print(f"✅ Response data exported to: telemetry_responses.csv")

print(f"\n📁 Exported Files Summary:")
print(f"   • {telemetry_file} - Full three-tier telemetry data")
print(f"   • {conv_file} - Multi-tier conversation history")
print(f"   • hybrid_foundry_summary_report.json - Enterprise routing summary")
print(f"   • telemetry_responses.csv - Three-tier response metrics")
print(f"   • hybrid_telemetry.log - Detailed routing logs")

print("\n🎉 Lab 6 Alternative Complete with Enhanced Analytics!")

## 🎉 Lab 6 Alternative Complete with Enhanced Routing Statistics!

### What You've Accomplished:
- ✅ Used `HybridFoundryAPIMRouter` from Lab 5 hybrid orchestration
- ✅ Implemented comprehensive telemetry with three-tier routing system
- ✅ **NEW**: Added routing statistics similar to lab6_observability_telemetry
- ✅ **NEW**: Enhanced performance analytics with multi-tier analysis
- ✅ **NEW**: Added optimization opportunity analysis
- ✅ **NEW**: Comprehensive stakeholder reporting
- ✅ Tracked ML-powered routing decisions (BERT/PHI)
- ✅ Monitored Local → APIM → Foundry Agent performance
- ✅ Created enterprise-grade analytics for three-tier routing

### Enhanced Analytics Features Added:

**📈 Performance Analytics Dashboard:**
- Global system metrics with runtime analysis
- Session-by-session performance breakdown
- Speed advantage calculations (local vs cloud)
- Multi-tier response time analysis

**🎯 Three-Tier Routing Efficiency Analysis:**
- Routing statistics from HybridFoundryAPIMRouter
- Source-specific response time analysis (LOCAL, APIM, FOUNDRY, AZURE)
- Hybrid efficiency metrics and time savings calculations
- Comprehensive routing distribution analysis

**🔧 Optimization Opportunity Analysis:**
- Routing decision pattern analysis
- Complexity score analysis by routing target
- Multi-tier response time performance analysis
- System capability recommendations
- Error rate and model switch frequency analysis

**📋 Enhanced Stakeholder Reporting:**
- Three-tier system executive summary
- Enterprise benefits quantification
- Success criteria assessment across all tiers
- ML-powered routing intelligence reporting
- Azure Foundry Agent capability tracking

### Key Technical Enhancements:
- **Routing Statistics Integration**: Direct integration with HybridFoundryAPIMRouter statistics
- **Multi-Source Analysis**: Separate tracking for LOCAL, APIM, FOUNDRY, and AZURE routing
- **ML Router Analytics**: BERT/PHI routing decision intelligence tracking
- **Enterprise Metrics**: APIM and Foundry-specific performance analytics
- **Optimization Insights**: Data-driven recommendations for routing improvements

### Advanced Capabilities:
- **ML-Powered Analysis**: Query analysis using BERT/PHI models with statistics
- **Enterprise Routing**: APIM integration performance monitoring
- **Agent Orchestration**: Azure AI Foundry Agent coordination analytics
- **Fallback Management**: Robust error handling and recovery statistics
- **Performance Optimization**: Real-time routing effectiveness analysis

### Files Created with Enhanced Analytics:
- `hybrid_foundry_telemetry_data.json` - Complete three-tier telemetry with routing stats
- `hybrid_foundry_conversation.json` - Multi-tier conversation history with metadata
- `hybrid_foundry_summary_report.json` - Enterprise routing summary with statistics
- `telemetry_responses.csv` - Enhanced response metrics with routing sources
- `hybrid_telemetry.log` - Detailed routing logs with analytics

### Enterprise Benefits with Analytics:
- **Cost Optimization**: Intelligent routing reduces cloud API costs with measurement
- **Performance Gains**: Local routing for fast responses with metrics
- **Quality Assurance**: APIM routing for complex enterprise queries with analytics
- **Advanced AI**: Foundry Agents for sophisticated reasoning with tracking
- **Full Observability**: Complete visibility into routing decisions with statistics
- **Data-Driven Optimization**: Analytics-based performance improvements

### Routing Statistics Similar to Lab6 Observability:
✅ **Performance Analytics**: Comprehensive dashboard with global and session metrics  
✅ **Routing Efficiency**: Multi-tier routing decision analysis  
✅ **Optimization Analysis**: Data-driven improvement recommendations  
✅ **Stakeholder Reporting**: Business-friendly analytics and insights  
✅ **Response Time Analysis**: Performance tracking across all routing tiers  
✅ **Error Pattern Analysis**: Comprehensive failure tracking and recovery metrics  

### Next Steps:
Your enterprise-grade three-tier hybrid orchestration system with comprehensive routing statistics and analytics is ready for production deployment and monitoring! The enhanced analytics provide the same depth of insights as the observability telemetry notebook, adapted for the advanced HybridFoundryAPIMRouter architecture. 🚀